# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
    # импорт библиотеки pandas
import pandas as pd       
    # чтение данных из файла и занесение данных в переменную
df = pd.read_csv("https://code.s3.yandex.net/datasets/data.csv")  
    # получение первичной сводной информации методом inof()
df.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Обратим внимание на общий вид датасета


In [2]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


В данных могут быть пропуски. Исследуем в более показательной форме:

In [3]:
df.isna().sum()   # получение информации о категориях данных с пропущенной информацией

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
    # формирование датафрейма с долей пропущеннах данных
missing_data = pd.DataFrame({'null_rate':df.isnull().sum()/len(df)})
    # форматирование представления в виде %
missing_data['null_rate'] = missing_data['null_rate'].map(lambda x: '{:.2%}'.format(x))
    # сортировка по убыванию и представление
missing_data.sort_values(by='null_rate', ascending=False)

,null_rate
days_employed,10.10%
total_income,10.10%
children,0.00%
dob_years,0.00%
education,0.00%
education_id,0.00%
family_status,0.00%
family_status_id,0.00%
gender,0.00%
income_type,0.00%


**Вывод**

1. в файле представленны данные по 12 категориям
2. общее количество строк равно 21525
3. типы данных: int64 (целочисленные), float64 (действительные), object (объектный тип)
4. имеются пропуски в категориях **"days_employed"** и **"total_income"**. Их количество равно между собой.
5. Причиной наличия пропущенных значений может служить:
    * человеческий фактор - ошибка ввода данных
    * машинный сбой при внесении данных
    * машинный сбой при выгрузке данных в файл

*Тем не менее, на этапе решения данной задачи возможность запросить восполнение недостающих данных отсутствует.*

Также из условия задачи подадобится описание данных:
    
### Описание данных
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита    

## Шаг 2. Предобработка данных

### Обработка пропусков

В соответствии с условиями задачи исследования необходимо ответить на следующие вопросы:
* "Есть ли зависимость между наличием детей и возвратом кредита в срок?"
* "Есть ли зависимость между семейным положением и возвратом кредита в срок?"
* "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?"

В связи с этим необходимо проводить исследование по следющим категориям предоставленных данных:
* **children** — количество детей в семье
* **family_status** — семейное положение
* **total_income** — ежемесячный доход

Следовательно, категория **days_employed**, в которой был обнаружен пропуск в данных, не будет участвовать в исслеловании, и каких-либо действий с данными в этой категории необходимости производить нет.

Далее, из трех категорий, по которым необходимо провести исслелование, данные отсутствуют только в категории **total_income**, поэтому перед исследованием необходимо восполнить данные в этой категории.

Будем использовать медиану, для того, чтобы уменьшить влияние восполненных данных на общий вывод. Но перед таким шагом необходимо убедиться, что в категории нет отрицательных значений, в противном случае медиана будет некорректно смещена к 0.


In [5]:
print(f'Минимальное значение в столбце total_income: {df.total_income.min()}')

Минимальное значение в столбце total_income: 20667.26379327158


In [6]:
df.total_income = df.total_income.fillna(df.total_income.median())
print(f'Количество пропусков в столбце total_income после замены пропусков: {df.total_income.isna().sum()}')

Количество пропусков в столбце total_income после замены пропусков: 0


<span style='background:yellow'>
Проведем заполнение недостающих значений в столбце "Доход" (total_income) медианными значениями в соответствии с типом занятости заемщиков
    </span>

<span style='background:yellow'>
Имеем следующие типы занятости
</span>

In [7]:
set(df.income_type)

{'безработный',
 'в декрете',
 'госслужащий',
 'компаньон',
 'пенсионер',
 'предприниматель',
 'сотрудник',
 'студент'}

<span style='background:yellow'>
Проведем упомянутю замену:</span>

In [8]:
for unique_income_type in df.income_type.unique():
    median = df.loc[df['income_type'] == unique_income_type, 'total_income'].median()
    print(f'Тип занятости: {unique_income_type} \nМедиана: {median}')
    print()
    df.loc[(df['total_income'].isna()) & (df['income_type'] == unique_income_type), 'total_income'] = median

Тип занятости: сотрудник 
Медиана: 145017.93753253992

Тип занятости: пенсионер 
Медиана: 128747.67556966442

Тип занятости: компаньон 
Медиана: 162401.35155456822

Тип занятости: госслужащий 
Медиана: 145017.93753253992

Тип занятости: безработный 
Медиана: 131339.7516762103

Тип занятости: предприниматель 
Медиана: 322090.5412398128

Тип занятости: студент 
Медиана: 98201.62531401133

Тип занятости: в декрете 
Медиана: 53829.13072905995



Проверим значения, которые принимают ячейки в исследуемых столбцах **children** и **family_status**

In [9]:
print(f'Значения полей для данных children: {df.children.unique()}')
df.family_status = df.family_status.str.lower()
print(f'Значения полей для данных family_status: {df.family_status.unique()}')
df.children.value_counts()

Значения полей для данных children: [ 1  0  3  2 -1  4 20  5]
Значения полей для данных family_status: ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Примечание: хотя значение количества детей "20" и является выбросом, тем не менее не является для РФ особенным, так как существуют семьи и с большим количеством детей, например, приемными.

С другой стороны тот факт, что количество семей именно с 20 детьми встречается 76 раз, говорит скорее, что мы имеем дело с человеческим фактором, где оператор, заносивший данные, небрежно нажимал на кнопку "2". В пользу этого говорит, что в датасете нет данных с "02" детьми. В связи с этим заменим эти данные на "2"


<span style='background:yellow'>
Также переведем отрицательные значения в положительные
</span>

In [10]:
df.loc[df['children'] == 20, 'children'] = 2
df.loc[df['children'] < 0, 'children'] = df.loc[df['children'] < 0, 'children'].abs()
df.children.value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**

1. В соответствии с условиями исследования были проверены данные на предмет наличия пропусков в категориях **children**, **family_status** и **total_income**
2. Данные в категориях **children** и **family_status** не имеют пропусков
3. Данные в категории **total_income** имели пропуски, которые, после того, как проверка не выявила в них отрицательных значений, были заменены на медианное значение

### Замена типа данных

1. В категории **children** выявлены отрицательные значения о количестве детей в семье, что не соответствует смыслу данной категории. Отрицательные значение будут заменены на положительные через модуль числа
2. В категории **total_income** присутствуют данные о доходе в виде действительных чисел (как положительных, так и отрицательных), что не соответствует:
    * расчетной валютной единице РФ
    * необходимой точности.
    * смыслу категории

        Эти данные будут заменены на натуральные.

In [11]:
df.children = abs(df.children)
df.total_income = df.total_income.astype('int')
print(f'Значения полей для данных children: {df.children.unique()}')
print('Значения месячного дохода в натуральных единицах по возрастанию: ')
print(df.total_income.sort_values(ascending=True).head())

Значения полей для данных children: [1 0 3 2 4 5]
Значения месячного дохода в натуральных единицах по возрастанию: 
14585    20667
13006    21205
16174    21367
1598     21695
14276    21895
Name: total_income, dtype: int32


**Вывод**

Проведено преобразование данных в столбцах **children** и **total_income** к неотрицательному целочисленному виду

### Обработка дубликатов

<span style='background:yellow'>
Проведем проверку данных на явные дубликаты
</span>

In [12]:
print(f'Количество дубликатов: {len(df[df.duplicated()])}') 
print(f'Длина датасета: {len(df)}')

Количество дубликатов: 54
Длина датасета: 21525


<span style='background:yellow'>
Очистим данные от явных дубликатов
</span>

In [13]:
df.drop_duplicates(inplace = True)
df.reset_index(drop=True) 
print(f'Длина датасета после очистки: {len(df)}')
df[df.duplicated()]

Длина датасета после очистки: 21471


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Обратим внимание на следующие категориальные (дискретные) данные, присутствующие в датасете и участвующие в исследовании: **family_status** и **purpose**

In [15]:
print('\033[1m' + 'Подсчет значений по категории: ' + '\033[0m' + 'family_status')
print('-'*44)
print(df.family_status.value_counts())
print('\n')
print('\033[1m' + 'Подсчет значений по категории: ' + '\033[0m' + 'purpose')
print('-'*44)
print(df.purpose.value_counts())

Подсчет значений по категории: family_status
--------------------------------------------
женат / замужем          12344
гражданский брак          4163
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


Подсчет значений по категории: purpose
--------------------------------------------
свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью      

**Вывод**

1. Категория **family_status** не вызывают вопросов с точки зрения дубликатов.
2. Категория **purpose** должна пройти лемматизацию, так как многие значения имеют одинаковую смысловую нагрузку, но выражены отличными друг от друга словосочетаниями

### Лемматизация

Предварительно, для исключения человеческого фактора, приведем значения в категории **purpose** к нижнему регистру через
использование метода ***str.lower()***

In [16]:
df.purpose = df.purpose.str.lower()
df.purpose.value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [17]:
from pymystem3 import Mystem # импортирование библиотеки
m = Mystem() 
lemmas = m.lemmatize(' '.join(df.purpose.unique()))  # создание списка лемм на основе категории
set(lemmas)

{'\n',
 ' ',
 'автомобиль',
 'высокий',
 'для',
 'дополнительный',
 'жилой',
 'жилье',
 'заниматься',
 'коммерческий',
 'на',
 'недвижимость',
 'образование',
 'операция',
 'подержать',
 'покупка',
 'получение',
 'приобретение',
 'проведение',
 'профильный',
 'ремонт',
 'с',
 'свадьба',
 'свой',
 'сдача',
 'сделка',
 'семья',
 'со',
 'собственный',
 'строительство',
 'сыграть'}

Исходя из сопоставления уникальных значений в списке лемм и уникальных значений категории **purpose**, можно сделать вывод, что направлениями кредита являются:
* *недвижимость / жилье*,
* *свадьба*,
* *покупка автомобиля*
* *образование*

Создадим новый столбец / категорию с унифицированными значениями, соответствующими категории **purpose**. Назовем ее **purpose_unified**

In [18]:
test_list = [['недвижим', 'жиль'], ['свадьб'], ['автомоб'], ['образова']]
result_list = ['недвижимость', 'свадьба', 'авто', 'образование']

def duplicates_unification(not_unified):
    for i in range(len(test_list)):
        result = [k for k in test_list[i] if(k in not_unified)]  # проверка содержания опорного элемента в переменной ф-ции
        if result:
            return result_list[i]                                # возвращаем значения и списка результатов
    
df['purpose_unified'] = df['purpose'].apply(duplicates_unification)  # применяем ф-цию к столбцу

df.purpose_unified.value_counts()               # проверка результата в категории 'purpose'

недвижимость    10814
авто             4308
образование      4014
свадьба          2335
Name: purpose_unified, dtype: int64

In [19]:
    # формирование датафрейма с долей цели кредита
purpose_rate = pd.DataFrame({'purpose_unified':df.purpose_unified.value_counts()/len(df)})
    # форматирование представления в виде %
purpose_rate['purpose_unified'] = purpose_rate['purpose_unified'].map(lambda x: '{:.2%}'.format(x))
    # сортировка по убыванию и представление
purpose_rate.sort_values(by='purpose_unified', ascending=False)

,purpose_unified
недвижимость,50.37%
авто,20.06%
образование,18.69%
свадьба,10.88%


**Вывод**

* Проверены и обработаны дубликаты в категориях: **family_status**, **income_type** и **purpose**
* На основе лемматизации категория **purpose** создана категория **purpose_unified** с унифицированными значениями, которые по смыслу сооответствуют **purpose**, что позволило избавиться от неявных дубликатов и получить чистые данные о целях кредита
* Выявлены следующие цели кредита по популярности: **недвижимость**	(50.36%), **авто** (20.05%), **образование** (18.69%) и **свадьба** (10.91%) *(результаты указаны в удельном выражении от общего числа кредитов в порядке убывания без привязки к денежному выражению)*

### Категоризация данных

В соответствии с условием ислледования необходимо провести категоризацию данных из следующих категорий: **family_status**, **total_income** и **children**

Проверим значения категории **debt**

In [20]:
df.debt.unique()

array([0, 1], dtype=int64)

1. Ранее было выяснено, что категория **debt** не имеет пропусков данных
2. Данная категория содержит только два значения: **0** и **1**
3. Исходя из того, что **True = 1**, а **False = 0**, получаем, что **0** - отсутствие задолженности, а **1** - присутствие задолженности, и исходя из этого продолжим исследование

Из необходимых категорий начнем с семейного положения **family_status**

Создадим новый дата фрейм на основе имеющегося путем группировки

In [21]:
debt_by_family_status = pd.DataFrame()
debt_by_family_status['family_status_sum'] = df.groupby('family_status')['debt'].sum()
debt_by_family_status['family_status_count'] = df.groupby('family_status')['debt'].count()
debt_by_family_status['family_status_debt_rate'] = debt_by_family_status['family_status_sum'] / debt_by_family_status['family_status_count'] 
debt_by_family_status['family_status_debt_rate'] = debt_by_family_status['family_status_debt_rate'].map(lambda x: '{:.2%}'.format(x))
debt_by_family_status.sort_values('family_status_debt_rate', ascending = False)

,family_status_sum,family_status_count,family_status_debt_rate
family_status,,,
не женат / не замужем,274,2810,9.75%
гражданский брак,388,4163,9.32%
женат / замужем,931,12344,7.54%
в разводе,85,1195,7.11%
вдовец / вдова,63,959,6.57%


Аналогичным образом поступим в части ислледования зависимости доли возвращенных не в срок кредитов от количества детей в семье (категория **children**).

In [22]:
debt_by_children = pd.DataFrame()
debt_by_children['children_sum'] = df.groupby('children')['debt'].sum()
debt_by_children['children_count'] = df.groupby('children')['debt'].count()
debt_by_children['children_debt_rate'] = debt_by_children['children_sum'] / debt_by_children['children_count'] 
debt_by_children['children_debt_rate'] = debt_by_children['children_debt_rate'].map(lambda x: '{:.2%}'.format(x))
debt_by_children.sort_values('children_debt_rate', ascending = False)

,children_sum,children_count,children_debt_rate
children,,,
4,4,41,9.76%
2,202,2128,9.49%
1,445,4856,9.16%
3,27,330,8.18%
0,1063,14107,7.54%
5,0,9,0.00%


Аналогичным образом поступим в части ислледования зависимости доли возвращенных не в срок кредитов от цели кредита (категория **purpose**).

In [23]:
debt_by_purpose = pd.DataFrame()
debt_by_purpose['purpose_sum'] = df.groupby('purpose_unified')['debt'].sum()
debt_by_purpose['purpose_count'] = df.groupby('purpose_unified')['debt'].count()
debt_by_purpose['purpose_debt_rate'] = debt_by_purpose['purpose_sum'] / debt_by_purpose['purpose_count'] 
debt_by_purpose['purpose_debt_rate'] = debt_by_purpose['purpose_debt_rate'].map(lambda x: '{:.2%}'.format(x))
debt_by_purpose.sort_values('purpose_debt_rate', ascending = False)

,purpose_sum,purpose_count,purpose_debt_rate
purpose_unified,,,
авто,403,4308,9.35%
образование,370,4014,9.22%
свадьба,186,2335,7.97%
недвижимость,782,10814,7.23%


Проверим заявленный минимальный доход заемщика

In [24]:
df.total_income.min()

20667

1. Необходимо разбить диапазон измерения получаемого дохода на некоторые равные интервалы. Таким образом мы сможем оценить в каждом из них долю невозвращенных кредитов
2. При выборке длинны такого интервала необходимо учитывать то, что при его увеличении количество частей, на которые бьется сам диапазон дохода, уменьшается, но в это же время сглаживается искомый параметр, а также больше измерений попадает в каждый интервал. И наоборот, при уменьшении такого интервала, количество измерений в попадающих в него, уменьшается
3. Рассмотрим интервал биения **total_income** в 100,000р.
4. Возьмем минимально необходимое количесвто измерений в получаемых интервалах не менее 30 (в противном случае мы не сможем доверять полученному показателю доли невозращенных в срок кредитов, так как увеличивается вероятность биения искомого показателя на малых объемах исследования)

In [25]:
threshold = 100000

df['total_income_interval'] = df.total_income // threshold * threshold
df_total_income = pd.DataFrame()
df_total_income['sum'] = df.groupby('total_income_interval')['debt'].sum()
df_total_income['count'] = df.groupby('total_income_interval')['debt'].count()

df_total_income['rate'] = df_total_income['sum'] / df_total_income['count']
df_total_income['rate'] = df_total_income['rate'].map(lambda x: '{:.2%}'.format(x))
#pd.set_option('display.max_rows', 100)
#df_total_income.tail(10)
df_limited = df_total_income[df_total_income['count'] > 30]
df_limited

,sum,count,rate
total_income_interval,,,
0,354,4463,7.93%
100000,1029,11942,8.62%
200000,252,3584,7.03%
300000,75,954,7.86%
400000,17,306,5.56%
500000,6,113,5.31%
600000,3,48,6.25%


**Вывод**

Проведена категоризация данных с рассчетом целевых показателей задания

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

**Вывод**

Заемщики с детьми незначительно хуже возвращают кредит, чем бездетные. Для заемщиков с детьми в количестве 5 данных недостаточно.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

**Вывод**

По результатам исследования выявлено, что такие категории как **«не женат / не замужем»** и **«гражданский брак»** хуже возвращают кредит, чем категории **«женат / замужем»** и **«в разводе»**. Лучше всего возвращают кредит заемщики из категории **«вдовец / вдова»**

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

**Вывод**

Заемщики с доходом от 400,000р лучше возвращают кредит, чем те, которые имеют доход менее данной суммы. Данные для заемщиков с доходом менее 700,000р являются непоказательными.

- Как разные цели кредита влияют на его возврат в срок?

**Вывод**

Заемщики с целю **«свадьба»** **и «недвижимость»** лучше возвращают кредиты, чем в категориях **«авто»** и **«образование»**

## Шаг 4. Общий вывод

В результате исследование получены данные о зависимости возврата кредита от категорий заемщиков с заданными статусами и получены следующие результаты:
* семейное положение:

<span style='background:yellow'>По результатам исследования выявлено, что такие категории как **«не женат / не замужем»** и **«гражданский брак»** хуже возвращают кредит, чем категории **«женат / замужем»** и **«в разводе»**. Лучше всего возвращают кредит заемщики из категории **«вдовец / вдова»**
</span>


* уровень дохода:

<span style='background:yellow'> Заемщики с доходом от 400,000р лучше возвращают кредит, чем те, которые имеют доход менее данной суммы. Данные для заемщиков с доходом менее 700,000р являются непоказательными.</span>



* количество детей:

<span style='background:yellow'>Заемщики с детьми незначительно хуже возвращают кредит, чем бездетные. Для заемщиков с детьми в количестве 5 данных недостаточно.
</span>


* цели кредита:

<span style='background:yellow'>Заемщики с целю **«свадьба»** **и «недвижимость»** лучше возвращают кредиты, чем в категориях **«авто»** и **«образование»**
</span>